In [1]:
import re
import codecs
import numpy as np
import pickle
from numpy import zeros, int8, log
from pyvi import ViTokenizer
from pylab import random

In [63]:


class PLSA:
    def __init__(self, K, maxIteration=30, threshold=10.0, topicWordsNum=5):
        self.K = K  # Số chủ đề
        self.maxIteration = maxIteration  # Số lần lặp tối đa
        self.threshold = threshold  # Ngưỡng dừng
        self.topicWordsNum = topicWordsNum  # Số từ hàng đầu trong mỗi chủ đề
        self.word2id = {}  
        self.id2word = {}  
        self.Pz = zeros(self.K)  # Xác suất của các chủ đề
        self.lamda = None  # Ma trận lambda P(d|z)
        self.theta = None  # Ma trận theta P(w|z)
        self.p = None  # Ma trận xác suất trung gian P(z|d,w)
        self.stopwordsFilePath='stopwords.dic'  
    
    def preprocessing(self, dataset):
        """
        Xử lý trước tài liệu, bao gồm tokenization, loại bỏ stopwords và xây dựng
        word2id, id2word và ma trận tần suất từ.
        """
        # Load stopwords
        file = codecs.open(self.stopwordsFilePath, 'r', 'utf-8')
        self.stopwords = [line.lower().strip() for line in file]
        file.close()

        documents = dataset
        N = len(documents)  # Số tài liệu
        wordCounts = []
        currentId = 0

        # Xây dựng word2id và id2word, đếm số lượng từ
        for document in documents:
            segList = ViTokenizer.tokenize(document).split()
            wordCount = {}
            for word in segList:
                word = word.lower().strip()
                if len(word) > 1 and not re.search('[0-9]', word) and word not in self.stopwords:
                    if word not in self.word2id:
                        self.word2id[word] = currentId
                        self.id2word[currentId] = word
                        currentId += 1
                    if word in wordCount:
                        wordCount[word] += 1
                    else:
                        wordCount[word] = 1
            wordCounts.append(wordCount)

        M = len(self.word2id)  # Số từ
        X = zeros([N, M], int8)
        for word in self.word2id.keys():
            j = self.word2id[word]
            for i in range(0, N):
                if word in wordCounts[i]:
                    X[i, j] = wordCounts[i][word]

        return N, M, X

    def initializeParameters(self, N, M):
        self.lamda = random([N, self.K])  # P(d|z)
        self.theta = random([self.K, M])  # P(w|z)
        self.Pz = random([self.K])  # P(z)
        self.Pz /= sum(self.Pz)  # Chuẩn hóa Pz để tổng bằng 1

        self.p = np.zeros((N, M, self.K))  # Ma trận xác suất P(z|d,w)

        # Chuẩn hóa lambda và theta
        for i in range(0, N):
            self.lamda[i, :] /= sum(self.lamda[i, :])

        for i in range(0, self.K):
            self.theta[i, :] /= sum(self.theta[i, :])

    def EStep(self, N, M, X):
        """
        Thực hiện bước E: Tính toán ma trận xác suất P(z|d,w).
        """
        for i in range(0, N):
            for j in range(0, M):
                denominator = np.sum(self.Pz[k] * self.lamda[i,k] * self.theta[k,j] for k in range(self.K))
                if denominator == 0:
                    self.p[i,j,:] = 1.0/self.K
                else:
                    for k in range(self.K):
                        self.p[i,j,k] = (self.Pz[k] * self.lamda[i,k] * self.theta[k,j]) / denominator    

    def MStep(self, N, M, X):
        # Cập nhật theta (P(w|z))
        for k in range(0, self.K):
            for j in range(0, M):
                self.theta[k,j] = np.sum(X[i,j] * self.p[i,j,k] for i in range(N))
            self.theta[k, :] /= np.sum(self.theta[k, :])

        # Cập nhật lambda (P(d|z))
        for k in range(self.K):
            for i in range(N):
                self.lamda[i,k] = np.sum(X[i,j] * self.p[i,j,k] for j in range(M))
            self.lamda[:, k] /= np.sum(self.lamda[:, k])

        for k in range(self.K):
            self.Pz[k] = np.sum(self.p[i,j,k] * X[i,j] for i in range(N) for j in range(M))

        # Chuẩn hóa Pz: chia cho tổng số lần xuất hiện của tất cả các từ và tài liệu
        total = np.sum(X)
        self.Pz /= total


        
    def LogLikelihood(self, N, M, X):
        loglikelihood = 0
        for i in range(0, N):
            for j in range(0, M):
                tmp = 0
                for k in range(0, self.K):
                    tmp += self.theta[k, j] * self.lamda[i, k] * self.Pz[k]
                if tmp > 0:
                    loglikelihood += X[i, j] * log(tmp)
        return loglikelihood

    def train(self, dataset):
        N, M, X = self.preprocessing(dataset)
        self.initializeParameters(N, M)

        oldLoglikelihood = 1
        newLoglikelihood = 1
        # print("training")
        for i in range(0, self.maxIteration):
            self.EStep(N, M, X)
            self.MStep(N, M, X)
            # print(self.Pz)
            newLoglikelihood = self.LogLikelihood(N, M, X)
            if oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < self.threshold:
                break
            oldLoglikelihood = newLoglikelihood
            # print(newLoglikelihood)

        topic = self.get_top_words()
        return self.p, self.Pz, self.lamda, self.theta, topic, self.id2word

    def get_top_words(self):
        topic = []
        for i in range(0, self.K):
            topicword = []
            ids = self.theta[i, :].argsort()
            for j in ids:
                topicword.insert(0, self.id2word[j])
            tmp = ''
            for word in topicword[0:min(self.topicWordsNum, len(topicword))]:
                tmp += word + ' '
            topic.append(tmp)
        return topic


    def test(self, data_test):
        segList= ViTokenizer.tokenize(data_test[0])    
        xtest = zeros(len(self.id2word))
        for word in segList.split(' '):
            word = word.lower().strip()
            # print(word)
            for i in range(len(self.id2word)):
                if word == self.id2word[i]:
                    xtest[i] += 1

        z = []
        for k in range(self.K):
            p_new = 1
            for i in range(len(xtest)):
                if xtest[i] != 0:
                    if self.theta[k,i] > 1e-6:
                        p_new *= self.theta[k,i] * xtest[i]
                        # print(p_new)
            if p_new == 1:
                z.append(0)
            else:
                p_new *= self.Pz[k]
                z.append(p_new)
        # print(z)

        if all(i == 0 for i in z):
            return "Khong thuoc chu de nao trong cac chu de tren" ,z      
        else:
            main_topic = z.index(min(z))
            return main_topic, z



    def save_model(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump({
                'Pz': self.Pz,
                'theta': self.theta,
                'lamda': self.lamda,
                'id2word': self.id2word,
                'word2id': self.word2id,
            }, f)

    def load_model(self, filename):
        with open(filename, 'rb') as f:
            params = pickle.load(f)
            self.Pz = params['Pz']
            self.theta = params['theta']
            self.lamda = params['lamda']
            self.id2word = params['id2word']
            self.word2id = params['word2id']
        topic = self.get_top_words()
        self.K = len(self.Pz)
        return self.p, self.Pz, self.lamda, self.theta, topic, self.id2word




In [64]:
file = codecs.open('data_test1.txt', 'r', 'utf-8')
test_data = [document.lower().strip() for document in file] 
file.close()
test_data[0]

'chiều 4/10, bộ công an đã tổ chức họp báo thông báo tình hình, kết quả công tác công an quý iii năm 2024. tại họp báo, đại diện các cục nghiệp vụ của bộ đã thông tin về tiến độ điều tra vụ án xảy ra tại công ty trách nhiệm hữu hạn cây xanh công minh (công ty công minh), vụ án tập đoàn phúc sơn, vụ tập đoàn thuận an và vụ án liên quan đến dự án sài gòn đại ninh. phát biểu tại họp báo, đại diện cơ quan an ninh điều tra, bộ công an cho biết, ngày 8/5/2024, cơ quan an ninh điều tra, bộ công an đã khởi tố vụ án "vi phạm quy định về đấu thầu gây hậu quả nghiêm trọng, đưa và nhận hối lộ" xảy ra tại công ty trách nhiệm hữu hạn cây xanh công minh và các tỉnh, thành phố. đến nay vụ án chưa khởi tố bị can. qua điều tra, cơ quan an ninh điều tra xác định, công ty công minh đã thông đồng với các chủ đầu tư để được tham gia vào việc xây dựng giá gói thầu từ giai đoạn lập dự án, sau khi giá hợp đồng được phê duyệt, công ty công minh sẽ được chỉ định trúng thầu hoặc sử dụng các pháp nhân khác trong h

In [65]:
plsa = PLSA(K=3, maxIteration=70,threshold=5.0, topicWordsNum=4)
p, Pz, lamda, theta, topic, id2w = plsa.load_model('D:\CDIO\CDIO4\PLSA_online\saved_plsa_model1.pkl')
file = codecs.open('data_test1.txt', 'r', 'utf-8')
test_data = [document.lower().strip() for document in file] 
file.close()

m,z= plsa.test(test_data)
print(z)
print("Topic: ",m)
print("Pz: ",Pz)
print(topic)

[4.637853804546126e-22, 1.5792503664079947e-19, 5.1254977029597224e-284]
Topic:  2
Pz:  [0.2415112  0.33132817 0.42716063]
['giáo_dục hỗ_trợ cơ_sở đồng ', 'đội_tuyển cầu_thủ về hlv ', 'điều_tra vụ về án ']


In [32]:
plsa = PLSA(K=3, maxIteration=70,threshold=5.0, topicWordsNum=4)
p, Pz, lamda, theta, topic, id2w = plsa.load_model('D:\CDIO\CDIO4\PLSA_online\saved_plsa_model1.pkl')
file = codecs.open('data_test1.txt', 'r', 'utf-8')
test_data = [document.lower().strip() for document in file] 
file.close()

m,z= plsa.test(test_data)
print(z)
print("Topic: ",m)
print("Pz: ",Pz)
print(topic)

[1.0543116811766803e-135, 1.423907354560807e-17, 1.2914493213546264e-20]
Topic:  1
Pz:  [0.2415112  0.33132817 0.42716063]
['giáo_dục hỗ_trợ cơ_sở đồng ', 'đội_tuyển cầu_thủ về hlv ', 'điều_tra vụ về án ']


In [4]:

file = codecs.open('data_train.txt', 'r', 'utf-8')
train_data = [document.strip() for document in file] 
file.close()


model = PLSA(K=3, maxIteration=70,threshold=5.0, topicWordsNum=4)

p, Pz, lamda, theta, wordTop, id2w = model.train(dataset=train_data)



print("Pz \n")
print(Pz)
print(wordTop)

file = codecs.open('data_test.txt', 'r', 'utf-8')
test_data = [document.lower().strip() for document in file] 
file.close()

m,z= model.test(test_data)
print(z)
print("Topic: ",m)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12604\2356869173.py:78: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  denominator = np.sum(self.Pz[k] * self.lamda[i,k] * self.theta[k,j] for k in range(self.K))
C:\Users\Admin\AppData\Local\Temp\ipykernel_12604\2356869173.py:89: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.theta[k,j] = np.sum(X[i,j] * self.p[i,j,k] for i in range(N))
C:\Users\Admin\AppData\Local\Temp\ipykernel_12604\2356869173.py:95: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  self.lamda[i,k] = np.sum(X[i,j] * self.p[i,j,k] for j in range(M))
C:\Users\Admin\AppData\Loca

Pz 

[0.3168027  0.38615648 0.29704082]
['quán ăn phở món ', 'trường học_sinh nước điểm ', 'đại_học trường thi xét ']
0.01664118139968115
8.01374526947486e-05
3.6944046783935264e-07
1.039964587055965e-09
2.7553161999063426e-13
6.678823121431074e-16
1.794646901276453e-18
5.073194624133819e-21
3.010457617064886e-23
1.864974704323312e-25
1.1838964121715627e-27
6.330713592954006e-30
7.099840039084437e-32
1.6249854094113752e-34
4.175958116619465e-38
4.2063256557151386e-41
7.508799524482802e-43
4.587524020805619e-45
1.414725226333067e-47
1.7515508180487887e-50
1.4178219655757795e-52
8.479640095697416e-56
4.358269517654424e-59
6.317799458187482e-62
2.1195773711178357e-63
3.941061510823731e-66
1.2373977176478792e-70
6.109309745392884e-74
5.939036435606178e-77
3.1524196856382036e-80
1.6581541509238122e-83
9.818157048344387e-86
3.1212357704091375e-88
2.3390767908196594e-91
4.406863846341267e-94
3.8727231960210876e-97
7.582583056955337e-100
9.71903131283336e-103
2.497638899580044e-106
7.856145078